# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/My\ Drive/maskrcnn-nnfl-github

/content/drive/My Drive/maskrcnn-nnfl-github


In [0]:
%tensorflow_version 1.x
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)
#tf.disable_v2_behavior()

# Root directory of the project
#ROOT_DIR = os.path.abspath("")
ROOT_DIR = os.path.abspath("/content/drive/My Drive/maskrcnn-nnfl-github/Mask_RCNN") # Aatman's Path
# ROOT_DIR = os.path.abspath("/content/drive/.shortcut-targets-by-id/18VderTbR8Ygk4oZ9m8Upm57jX9-b5oo6/repos/Mask_RCNN") # KUNAL'S path
#ROOT_DIR = os.path.abspath("/content/drive/.shortcut-targets-by-id/18VderTbR8Ygk4oZ9m8Upm57jX9-b5oo6/repos/Mask_RCNN") # Himank's path
print(ROOT_DIR)
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "weights.h5")

IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [11]:
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## model creation and weights addition

In [12]:
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
# training_weights = os.path.join(ROOT_DIR, "official_logs/training_1/mask_rcnn_coco_0125.h5")
# model.load_weights(training_weights, by_name=True)
model.load_weights(COCO_MODEL_PATH, by_name=True)

 
123456789
 
scores= Tensor("ROI_1/strided_slice:0", shape=(?, ?), dtype=float32)
ix= Tensor("ROI_1/top_anchors:1", shape=(?, ?), dtype=int32)
scores= Tensor("ROI_1/ExpandDims:0", shape=(1, ?), dtype=float32)
deltas= Tensor("ROI_1/ExpandDims_1:0", shape=(1, ?, 4), dtype=float32)
pre_nms_anchors= Tensor("ROI_1/ExpandDims_2:0", shape=(1, ?, 4), dtype=float32)
boxes= Tensor("ROI_1/ExpandDims_3:0", shape=(1, ?, 4), dtype=float32)
boxes= Tensor("ROI_1/ExpandDims_4:0", shape=(1, ?, 4), dtype=float32)
proposals= Tensor("ROI_1/ExpandDims_5:0", shape=(1, ?, 4), dtype=float32)


In [0]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# inference (visualization left)

In [0]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[0]))

# Run detection
results = model.detect([image], verbose=1)